<a href="https://colab.research.google.com/github/RonaldTuna/TensorIoTAssignment/blob/main/TensorIoT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEPS:**

Download the data files from here - http://jmcauley.ucsd.edu/data/amazon/links.html

Apache spark tools locally and necessary tools

Download a review file with a million reviews

Using Jupyter notebook create a program to read the million reviews and get the following

transform date to MM-DD-YYYY format

Save the data into a table (postgres/sql server)
Save the output as a Parquet file


Upload code to Github  and complete Readme.md which anyone can understand

Send Github link to HR



Skills Learning / Tools Used:

    postgres
    workbench/J
    Jupyter notebook
    Apache spark
    Data frame transforms
    









In [1]:
pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

I downloaded the reviews for Android Applications, which contains 2,638,173 reviews.

Load reviews into pyspark dataframe... or ADD?

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
df = spark.read.json('/content/reviews_Apps_for_Android.json.gz')

Transform data from MM DD, YYYY to MM-DD-YYYY by using to_date function.

Replace old column with new column containing reformatted dates.

In [13]:
df = df.withColumn('newTime', to_date('reviewTime', format='MM d, yyyy'))
df = df.drop("reviewTime")
df = df.withColumnRenamed("newTime","reviewTime")
df.show()


Save data into SQL table/server

+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+----------+
|      asin| helpful|overall|          reviewText|    reviewerID|        reviewerName|             summary|unixReviewTime|reviewTime|
+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+----------+
|B004A9SDD8|  [0, 0]|    5.0|Glad to finally s...| AUI0OLXAB3KKT|          A Customer|        Great app!!!|    1301184000|2011-03-27|
|B004A9SDD8|[12, 14]|    5.0|this app works gr...|A1ZUSQ3TC3EC4C|           A. Lissak|        Kid loves it|    1321574400|2011-11-18|
|B004A9SDD8|  [0, 0]|    4.0|We love these mon...| AC05OAXD72X1V|               Allie| Love these monkeys!|    1367366400|2013-05-01|
|B004A9SDD8|  [0, 2]|    5.0|cannot get my kin...|A2RVMFOKBVM21I|     Amazon Customer|fun fun for toddlers|    1350172800|2012-10-14|
|B004A9SDD8|  [1, 3]|    1.0|I start this app ...|A3NBSRGUWQGC

NameError: ignored

Save data into SQL table/server
Save file as parquet file.

> Indented block



In [ ]:
df.write.parquet("Amazon_reviews.parquet")